Weather data preparation 

In [0]:
import pandas as pd
import numpy as np

In [7]:
#Read master csv file considering first column in the dataset as the index column
df_master = pd.read_csv("master.csv", index_col=0)

#df_master = df_master.drop(['County Centroid', 'Disease_INFLUENZA_A', 'Disease_INFLUENZA_B', 'Disease_INFLUENZA_UNSPECIFIED', 'fluSymptoms', 'soreThroat', 'VaxRate', 'Population', 'GDP_change'], axis=1)   #Serves no purpose - drop the columns just for accomodating all the required columns in the same sheet (easy for debugging)
print((df_master['County'].unique()))
df_master.head()

['ALBANY' 'ALLEGANY' 'BRONX' 'BROOME' 'CATTARAUGUS' 'CAYUGA' 'CHAUTAUQUA'
 'CHEMUNG' 'CHENANGO' 'CLINTON' 'COLUMBIA' 'CORTLAND' 'DELAWARE'
 'DUTCHESS' 'ERIE' 'ESSEX' 'FRANKLIN' 'FULTON' 'GENESEE' 'GREENE'
 'HAMILTON' 'HERKIMER' 'JEFFERSON' 'KINGS' 'LEWIS' 'LIVINGSTON' 'MADISON'
 'MONROE' 'MONTGOMERY' 'NASSAU' 'NEW YORK' 'NIAGARA' 'ONEIDA' 'ONONDAGA'
 'ONTARIO' 'ORANGE' 'ORLEANS' 'OSWEGO' 'OTSEGO' 'PUTNAM' 'QUEENS'
 'RENSSELAER' 'RICHMOND' 'ROCKLAND' 'SARATOGA' 'SCHENECTADY' 'SCHOHARIE'
 'SCHUYLER' 'SENECA' 'ST LAWRENCE' 'STEUBEN' 'SUFFOLK' 'SULLIVAN' 'TIOGA'
 'TOMPKINS' 'ULSTER' 'WARREN' 'WASHINGTON' 'WAYNE' 'WESTCHESTER' 'WYOMING'
 'YATES']


,Unnamed: 0.1,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,flu,fluSymptoms,fever,cough,soreThroat,Target,VaxRate
0,0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0,6.8,-,-,-,0.211869,0.174420,0.206388,0.112639,0.087600,27.0,NaN
1,1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0,6.8,-,-,-,0.334176,0.176985,0.138915,0.083486,0.067525,87.0,NaN
2,2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0,6.8,-,-,-,0.761100,0.657923,0.492075,0.156241,0.091104,77.0,NaN
3,3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0,6.8,-,-,-,0.849600,0.855000,0.575424,0.214376,0.078840,61.0,NaN
4,4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0,6.8,-,-,-,0.530469,0.738720,0.396900,0.246318,0.125195,29.0,NaN


In [0]:
#Works on a single county basis
county_name = 'PUTNAM'

In [18]:
#Read CSV file
county_name = 'ROCKLAND'
df_TPS_1 = pd.read_csv(county_name.title() + ".csv")
df_TPS_1.head()
df_TPS_1 = df_TPS_1.rename(columns={" Date": "DATE", " Precipitation": "PRCP", " MaxTemperature": "TMAX", " MinTemperature": "TMIN", " AvgTemperature": "TAVG", " Snowfall": "SNOW", " SnowDepth": "SNOWDepth"})
#print(df_TPS_1.head())

df_TPS_1['PRCP'] = pd.to_numeric(df_TPS_1['PRCP'], errors='coerce')
df_TPS_1['SNOW'] = pd.to_numeric(df_TPS_1['SNOW'], errors='coerce')
df_TPS_1['TMAX'] = pd.to_numeric(df_TPS_1['TMAX'], errors='coerce')
df_TPS_1['TMIN'] = pd.to_numeric(df_TPS_1['TMIN'], errors='coerce')
df_TPS_1['TAVG'] = pd.to_numeric(df_TPS_1['TAVG'], errors='coerce')
df_TPS_1['SNOWDepth'] = pd.to_numeric(df_TPS_1['SNOWDepth'], errors='coerce')

#print(df_TPS_1.applymap(np.isreal))
print(df_TPS_1.shape)
df_TPS_1.sample(2)
#df_TPS_1.to_csv("temp.csv")

(3743, 7)


,Date,TMAX,TMIN,TAVG,PRCP,SNOW,SNOWDepth
3707,2019-10-26,NaN,NaN,NaN,NaN,NaN,NaN
2500,2016-07-06,85.0,66.0,75.5,0.0,0.0,0.0


In [19]:
#Convert df_TPS['DATE'] to pandas Date format
df_TPS_1['Date'] = pd.to_datetime(df_TPS_1['Date'])
#Extract Week number from date
df_TPS_1['Week_Number'] = df_TPS_1['Date'].dt.week
#Extract Year number from date
df_TPS_1['Year'] = df_TPS_1['Date'].dt.year
df_TPS_1.fillna(df_TPS_1.groupby(df_TPS_1.Date.dt.month).transform('mean'))
#df_TPS_1.to_csv("temp2.csv")

#df_TPS_1 = df_TPS_1.drop(['Date'], axis = 1)
#print(df_TPS_1.sample(5))

#Create a dict to denote how a feature has to be grouped 
#Eg: PRCP : mean - PRCP features are averaged over complete week
agg_dict = {'PRCP'  : 'mean',
            'SNOW'  : 'mean',
            'TAVG'  : 'mean', 
            'TMAX'  : 'mean',
            'TMIN'  : 'mean',
            'SNOWDepth' : 'mean'}

#Group same Year and WeekNumnber as per the values in the agg_dict
df_TPS_1 = df_TPS_1.groupby(['Year','Week_Number']).agg(agg_dict)
#Year and Week_NUmber are in index format, convert them into columns
df_TPS_1.reset_index(inplace=True)
#Take mean of TMAX and TMIN to get TAVG (TMAX and TMIN here are already averages of the entire week)
#df_TPS['TAVG'] = (df_TPS['TMAX'] + df_TPS['TMIN'])/2
print(df_TPS_1.shape)
df_TPS_1.sample(2)

(537, 8)


,Year,Week_Number,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth
373,2016,42,0.24,0.0,60.071429,73.0,47.142857,0.0
416,2017,32,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#Creates a new dataframe by merging only the required county in master dataframe and the df_TPS dataframe for that county
#This new dataframe is only a partial dataframe that contains only the rows that have the mentioned county values
#Complete dataset has to be created by either merging or creating df_TPS for all counties and cerging all temp_df
temp_df = pd.merge(df_master.loc[df_master['County'] == county_name], df_TPS_1,  how='left', left_on=['year','CDC Week'], right_on = ['Year','Week_Number'])
#Drop the newly created Year and Week_Number columns as our master dataset already has these columns
temp_df = temp_df.drop(['Year', 'Week_Number'], axis = 1)
#temp_df.sample()
print(temp_df.shape)
master_updated = pd.concat([master_updated, temp_df])
print(master_updated.shape)
master_updated.sample(10)

(336, 28)
(20080, 28)


,Unnamed: 0.1,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,flu,fluSymptoms,fever,cough,soreThroat,Target,VaxRate,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth
326,10998,ONTARIO,36069,"(42.8566949, -77.3032768)",2019,16,04/20/2019,19,19,0,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,5.0,NaN,0.194286,0.00,45.071429,53.571429,36.571429,0.000000
121,1061,BROOME,36007,"(42.1619773, -75.830291)",2013,10,03/09/2013,11,6,5,0,7.8,8525008.0,-2.0,197911.0,0.007000,0.006300,0.344250,0.214500,0.000000,8.0,NaN,0.034000,0.46,31.000000,37.857143,24.142857,4.571429
297,1539,CATTARAUGUS,36009,"(42.2448527, -78.6810055)",2019,40,10/05/2019,0,0,0,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.105714,0.00,57.071429,67.000000,47.142857,0.000000
321,14689,SCHENECTADY,36093,"(42.8175421, -74.0435834)",2019,11,03/16/2019,104,85,8,11,-,-,-,-,NaN,NaN,NaN,NaN,NaN,141.0,NaN,0.020000,0.00,35.714286,46.285714,25.142857,2.571429
213,8533,MONROE,36055,"(43.174966, -77.639663)",2016,3,01/23/2016,10,9,0,1,4.7,-,-,743531.0,0.005486,0.004514,0.467894,0.354859,0.203885,22.0,0.575,0.037143,NaN,23.285714,29.571429,17.000000,NaN
256,12944,QUEENS,36081,"(40.7282, -73.7949)",2017,12,03/25/2017,163,39,123,1,4.0,-,-,2296865.0,0.004367,0.002157,0.403065,0.204733,0.289645,259.0,0.662,0.025000,0.00,42.642857,51.428571,33.857143,0.000000
161,14865,SCHOHARIE,36095,"(42.591294, -74.4381718)",2014,17,04/26/2014,0,0,0,0,6.7,665785.0,-0.4,31773.0,0.003577,0.003615,0.159495,0.114065,0.172838,0.0,0.631,NaN,NaN,NaN,NaN,NaN,NaN
26,6704,JEFFERSON,36045,"(44.0607, -75.9928)",2010,15,04/17/2010,0,0,0,0,9.3,-,-,116594.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN,0.061429,0.00,46.428571,56.428571,36.428571,0.000000
175,13535,RICHMOND,36085,"(40.5795, -74.1502)",2014,50,12/13/2014,2,1,1,0,7.4,11880328.0,5.4,472075.0,0.008933,0.005871,0.250733,0.289181,0.387464,10.0,0.627,0.190000,8.00,29.000000,32.714286,25.285714,8.000000
235,6309,HAMILTON,36041,"(43.6578786, -74.5024556)",2017,44,11/04/2017,0,0,0,0,7.4,-,-,4481.0,0.008624,0.001687,0.266254,0.429442,0.238162,0.0,0.337,0.471667,0.00,41.857143,49.428571,34.285714,0.000000


In [0]:
master_updated.to_csv("master.csv")

In [23]:
master_updated

,Unnamed: 0.1,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,flu,fluSymptoms,fever,cough,soreThroat,Target,VaxRate,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth
0,0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0,6.8,-,-,-,0.211869,0.174420,0.206388,0.112639,0.087600,27.0,NaN,0.058333,0.0,54.642857,61.285714,48.000000,0.0
1,1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0,6.8,-,-,-,0.334176,0.176985,0.138915,0.083486,0.067525,87.0,NaN,0.058571,0.0,52.000000,60.000000,44.000000,0.0
2,2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0,6.8,-,-,-,0.761100,0.657923,0.492075,0.156241,0.091104,77.0,NaN,0.041667,0.0,39.071429,48.285714,29.857143,0.0
3,3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0,6.8,-,-,-,0.849600,0.855000,0.575424,0.214376,0.078840,61.0,NaN,0.303333,0.0,48.000000,58.857143,37.142857,0.0
4,4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0,6.8,-,-,-,0.530469,0.738720,0.396900,0.246317,0.125195,29.0,NaN,0.212857,0.0,48.857143,56.285714,41.428571,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,14027,ROCKLAND,36087,"(41.1547851, -74.0247721)",2019,40,10/05/2019,2,1,1,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
332,14028,ROCKLAND,36087,"(41.1547851, -74.0247721)",2019,41,10/12/2019,2,1,1,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333,14029,ROCKLAND,36087,"(41.1547851, -74.0247721)",2019,42,10/19/2019,2,1,1,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,14030,ROCKLAND,36087,"(41.1547851, -74.0247721)",2019,43,10/26/2019,1,0,1,0,-,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
master_df.to_csv("master.csv")